# HuggingFace Custom Project 🤗
---

 GLUE dataset의 mnli task를 수행하는 프로젝트를 커스텀 프로젝트 형태로 진행해 본다.

#### environment setting

In [1]:
import os
import numpy as np
from argparse import ArgumentParser
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from transformers import BertTokenizer, TFBertForSequenceClassification, AutoConfig
from transformers import BartTokenizer, BartForSequenceClassification, AutoConfig
from transformers import RobertaTokenizerFast, TFRobertaForSequenceClassification
from dataclasses import asdict
from transformers.data.processors.utils import DataProcessor, InputExample, InputFeatures
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import warnings

In [2]:
tf.__version__

'2.4.1'

In [3]:
# initialization gpu ########################################################
import tensorflow as tf
def init_gpu():
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            # Currently, memory growth needs to be the same across GPUs
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs\n\n")
        except RuntimeError as e:
            # Memory growth must be set before GPUs have been initialized
            print(e, "\n\n")
        
init_gpu()

1 Physical GPUs, 1 Logical GPUs




In [4]:
workspace_path = os.path.join(os.getenv('HOME'), 'workspace/aiffel-gd-nlp/GD18_hugging_face_custom')
# data_dir_path = os.path.join(workspace_path, 'data')
# model_dir_path = os.path.join(workspace_path, 'models')

## Step 1. mnli 데이터셋 분석
---

- 데이터 불러오기

In [5]:
data, info = tfds.load('glue/mnli', with_info=True)
info.splits['train'].num_examples

INFO:absl:Load dataset info from /home/aiffel-dj1/tensorflow_datasets/glue/mnli/1.0.0
INFO:absl:Reusing dataset glue (/home/aiffel-dj1/tensorflow_datasets/glue/mnli/1.0.0)
INFO:absl:Constructing tf.data.Dataset glue for split None, from /home/aiffel-dj1/tensorflow_datasets/glue/mnli/1.0.0


392702

In [6]:
info

tfds.core.DatasetInfo(
    name='glue',
    full_name='glue/mnli/1.0.0',
    description="""
    GLUE, the General Language Understanding Evaluation benchmark
    (https://gluebenchmark.com/) is a collection of resources for training,
    evaluating, and analyzing natural language understanding systems.
    """,
    config_description="""
    The Multi-Genre Natural Language Inference Corpus is a crowdsourced
    collection of sentence pairs with textual entailment annotations. Given a premise sentence
    and a hypothesis sentence, the task is to predict whether the premise entails the hypothesis
    (entailment), contradicts the hypothesis (contradiction), or neither (neutral). The premise sentences are
    gathered from ten different sources, including transcribed speech, fiction, and government reports.
    We use the standard test set, for which we obtained private labels from the authors, and evaluate
    on both the matched (in-domain) and mismatched (cross-domain) section. We a

- 데이터 확인

In [7]:
data['train'].take(1)

<TakeDataset shapes: {hypothesis: (), idx: (), label: (), premise: ()}, types: {hypothesis: tf.string, idx: tf.int32, label: tf.int64, premise: tf.string}>

In [8]:
examples = data['train'].take(1)
for example in examples:
    hypothesis = example['hypothesis']
    idx = example['idx']
    label = example['label']
    premise = example['premise']
    print(hypothesis.numpy())
    print(idx.numpy())
    print(label.numpy())
    print(premise.numpy())

b'Meaningful partnerships with stakeholders is crucial.'
16399
1
b'In recognition of these tensions, LSC has worked diligently since 1995 to convey the expectations of the State Planning Initiative and to establish meaningful partnerships with stakeholders aimed at fostering a new symbiosis between the federal provider and recipients of legal services funding.'


## Step 2. MNLIProcessor클래스를 구현
---

In [9]:
class DataProcessor:
    """Base class for data converters for sequence classification data sets."""

    def get_example_from_tensor_dict(self, tensor_dict):
        """
        Gets an example from a dict with tensorflow tensors.

        Args:
            tensor_dict: Keys and values should match the corresponding Glue
                tensorflow_dataset examples.
        """
        raise NotImplementedError()

    def get_train_examples(self, data_dir):
        """Gets a collection of :class:`InputExample` for the train set."""
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        """Gets a collection of :class:`InputExample` for the dev set."""
        raise NotImplementedError()

    def get_test_examples(self, data_dir):
        """Gets a collection of :class:`InputExample` for the test set."""
        raise NotImplementedError()

    def get_labels(self):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()

    def tfds_map(self, example):
        """
        Some tensorflow_datasets datasets are not formatted the same way the GLUE datasets are. This method converts
        examples to the correct format.
        """
        if len(self.get_labels()) > 1:
            example.label = self.get_labels()[int(example.label)]
        return example

    @classmethod
    def _read_tsv(cls, input_file, quotechar=None):
        """Reads a tab separated value file."""
        with open(input_file, "r", encoding="utf-8-sig") as f:
            return list(csv.reader(f, delimiter="\t", quotechar=quotechar))

In [10]:
class MnliProcessor(DataProcessor):
    """Processor for the MultiNLI data set (GLUE version)."""

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        warnings.filterwarnings("ignore", category=DeprecationWarning) 

    def get_example_from_tensor_dict(self, tensor_dict):
        """See base class."""
        return InputExample(
            tensor_dict["idx"].numpy(),
            tensor_dict["premise"].numpy().decode("utf-8"),
            tensor_dict["hypothesis"].numpy().decode("utf-8"),
            str(tensor_dict["label"].numpy()),
        )

    def get_train_examples(self, data_dir):
        """See base class."""
        print("LOOKING AT {}".format(os.path.join(data_dir, "train.tsv")))
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "train.tsv")), "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "dev.tsv")), "dev")

    def get_test_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "test.tsv")), "test")
    
    def get_labels(self):
        """See base class."""
        return ["0", "1", "2"]

    def _create_examples(self, lines, set_type):
        """Creates examples for the training, dev and test sets."""
        examples = []
        for (i, line) in enumerate(lines):
            if i == 0:
                continue
            guid = f"{set_type}-{line[0]}"
            text_a = line[8]
            text_b = line[9]
            label = None if set_type.startswith("test") else line[-1]
            examples.append(InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
        return examples

In [11]:
class MnliMismatchedProcessor(MnliProcessor):
    """Processor for the MultiNLI Mismatched data set (GLUE version)."""

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "dev_mismatched.tsv")), "dev_mismatched")

    def get_test_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "test_mismatched.tsv")), "test_mismatched")

In [12]:
processor = MnliProcessor()

examples = data['train'].take(1)

for example in examples:
    print('------원본데이터------')
    print(example)  
    example = processor.get_example_from_tensor_dict(example)
    print('------processor 가공데이터------')
    print(example)

------원본데이터------
{'hypothesis': <tf.Tensor: shape=(), dtype=string, numpy=b'Meaningful partnerships with stakeholders is crucial.'>, 'idx': <tf.Tensor: shape=(), dtype=int32, numpy=16399>, 'label': <tf.Tensor: shape=(), dtype=int64, numpy=1>, 'premise': <tf.Tensor: shape=(), dtype=string, numpy=b'In recognition of these tensions, LSC has worked diligently since 1995 to convey the expectations of the State Planning Initiative and to establish meaningful partnerships with stakeholders aimed at fostering a new symbiosis between the federal provider and recipients of legal services funding.'>}
------processor 가공데이터------
InputExample(guid=16399, text_a='In recognition of these tensions, LSC has worked diligently since 1995 to convey the expectations of the State Planning Initiative and to establish meaningful partnerships with stakeholders aimed at fostering a new symbiosis between the federal provider and recipients of legal services funding.', text_b='Meaningful partnerships with stakeh

In [13]:
examples = (data['train'].take(1))
for example in examples:
    example = processor.get_example_from_tensor_dict(example)
    example = processor.tfds_map(example)
    print(example)

InputExample(guid=16399, text_a='In recognition of these tensions, LSC has worked diligently since 1995 to convey the expectations of the State Planning Initiative and to establish meaningful partnerships with stakeholders aimed at fostering a new symbiosis between the federal provider and recipients of legal services funding.', text_b='Meaningful partnerships with stakeholders is crucial.', label='1')


In [14]:
label_list = processor.get_labels()
label_list

['0', '1', '2']

In [15]:
label_map = {label: i for i, label in enumerate(label_list)}
label_map

{'0': 0, '1': 1, '2': 2}

## Step 3. 데이터셋 구성
---

In [16]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [17]:
def _glue_convert_examples_to_features(examples, tokenizer, max_length, processor, label_list=None, output_mode="claasification") :
    if max_length is None :
        max_length = tokenizer.max_len
    if label_list is None:
        label_list = processor.get_labels()
        print("Using label list %s" % (label_list))

    label_map = {label: i for i, label in enumerate(label_list)}
    labels = [label_map[example.label] for example in examples]

    batch_encoding = tokenizer(
        [(example.text_a, example.text_b) for example in examples],
        max_length=max_length,
        padding="max_length",
        truncation=True,
    )

    features = []
    for i in range(len(examples)):
        inputs = {k: batch_encoding[k][i] for k in batch_encoding}

        feature = InputFeatures(**inputs, label=labels[i])
        features.append(feature)

    for i, example in enumerate(examples[:5]):
        print("*** Example ***")
        print("guid: %s" % (example.guid))
        print("features: %s" % features[i])

    return features

In [18]:
def tf_glue_convert_examples_to_features(examples, tokenizer, max_length, processor, label_list=None, output_mode="classification") :
    """
    :param examples: tf.data.Dataset
    :param tokenizer: pretrained tokenizer
    :param max_length: example의 최대 길이(기본값 : tokenizer의 max_len)
    :param task: GLUE task 이름
    :param label_list: 라벨 리스트
    :param output_mode: "regression" or "classification"

    :return: task에 맞도록 feature가 구성된 tf.data.Dataset
    """
    examples = [processor.tfds_map(processor.get_example_from_tensor_dict(example)) for example in examples]
    features = _glue_convert_examples_to_features(examples, tokenizer, max_length, processor)
    label_type = tf.int64

    def gen():
        for ex in features:
            d = {k: v for k, v in asdict(ex).items() if v is not None}
            label = d.pop("label")
            yield (d, label)

    input_names = ["input_ids"] + tokenizer.model_input_names

    return tf.data.Dataset.from_generator(
        gen,
        ({k: tf.int32 for k in input_names}, label_type),
        ({k: tf.TensorShape([None]) for k in input_names}, tf.TensorShape([])),
    )

- 학습 데이터 생성

In [19]:
batch_size = 8
max_len = 128

In [20]:
# train 데이터셋
train_dataset = tf_glue_convert_examples_to_features(data['train'], tokenizer, max_length=max_len, processor=processor)
train_dataset_batch = train_dataset.shuffle(100).batch(batch_size).repeat(2)

Using label list ['0', '1', '2']
*** Example ***
guid: 16399
features: InputFeatures(input_ids=[101, 1999, 5038, 1997, 2122, 13136, 1010, 1048, 11020, 2038, 2499, 29454, 29206, 14626, 2144, 2786, 2000, 16636, 1996, 10908, 1997, 1996, 2110, 4041, 6349, 1998, 2000, 5323, 15902, 13797, 2007, 22859, 6461, 2012, 6469, 2075, 1037, 2047, 25353, 14905, 10735, 2483, 2090, 1996, 2976, 10802, 1998, 15991, 1997, 3423, 2578, 4804, 1012, 102, 15902, 13797, 2007, 22859, 2003, 10232, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [21]:
# validation 데이터셋
# validation_dataset_match = tf_glue_convert_examples_to_features(data['validation_matched'], tokenizer, max_length=max_len, processor=processor)
# validation_dataset_mismatch = tf_glue_convert_examples_to_features(data['validation_matched'], tokenizer, max_length=max_len, processor=processor_mismatched)
# validation_dataset = validation_dataset_match.concatenate(validation_dataset_mismatch)
validation_dataset = tf_glue_convert_examples_to_features(data['validation_matched'], tokenizer, max_length=max_len, processor=processor)
validation_dataset_batch = validation_dataset.shuffle(100).batch(batch_size)

Using label list ['0', '1', '2']
*** Example ***
guid: 6287
features: InputFeatures(input_ids=[101, 7910, 1011, 9616, 2821, 3398, 2035, 1996, 2111, 2005, 2157, 7910, 2166, 2030, 2242, 102, 3398, 7167, 1997, 2111, 2005, 1996, 2157, 2166, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [22]:
# test 데이터셋
# test_dataset_match = tf_glue_convert_examples_to_features(data['test_matched'], tokenizer, max_length=max_len, processor=processor)
# test_dataset_mismatch = tf_glue_convert_examples_to_features(data['test_mismatched'], tokenizer, max_length=max_len, processor=processor_mismatched)
# test_dataset = test_dataset_match.concatenate(test_dataset_mismatch)
test_dataset = tf_glue_convert_examples_to_features(data['test_matched'], tokenizer, max_length=max_len, processor=processor)
test_dataset_batch = test_dataset.shuffle(100).batch(batch_size)

Using label list ['0', '1', '2']
*** Example ***
guid: 5398
features: InputFeatures(input_ids=[101, 2092, 2003, 2045, 1037, 2210, 3751, 10216, 2017, 2404, 1999, 2045, 102, 2003, 2045, 1037, 10216, 2008, 2017, 2404, 1999, 2045, 1029, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

## Step 4. 모델 학습 및 평가
---

In [23]:
num_classes = len(processor.get_labels())

optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

## 아래의 값이 OOM을 유발하여 200으로 고정시켜 주었다.
steps_per_epoch = tf.data.experimental.cardinality(data['train']).numpy() // batch_size
steps_per_epoch = 200

### 모델 학습 1 - TFBertForSequenceClassification(bert-base-cased)

In [24]:
from transformers import TFBertForSequenceClassification
model_bert_base_cased = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_classes)
model_bert_base_cased.compile(optimizer=optimizer, loss=loss, metrics=['acc', 
                                                                       'sparse_categorical_accuracy'])
model_bert_base_cased.summary()

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  2307      
Total params: 109,484,547
Trainable params: 109,484,547
Non-trainable params: 0
_________________________________________________________________


In [25]:
# 이전 스텝에서 배치처리를 진행한 데이터셋(xxxx_dataset_batch)을 활용
model_bert_base_cased.fit(train_dataset_batch, epochs=3, steps_per_epoch=10000, 
                             validation_data=validation_dataset_batch)

Epoch 1/3
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


10000/10000 [==============================] - ETA: 0s - loss: 0.7167 - acc: 0.6930 - sparse_categorical_accuracy: 0.6930WARNING:tensorflow:The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


10000/10000 [==============================] - 1860s 185ms/step - loss: 0.7167 - acc: 0.6930 - sparse_categorical_accuracy: 0.6930 - val_loss: 0.5374 - val_acc: 0.7892 - val_sparse_categorical_accuracy: 0.7892
Epoch 2/3
10000/10000 [==============================] - 1816s 182ms/step - loss: 0.5560 - acc: 0.7779 - sparse_categorical_accuracy: 0.7779 - val_loss: 0.5338 - val_acc: 0.7882 - val_sparse_categorical_accuracy: 0.7882
Epoch 3/3
10000/10000 [==============================] - 1892s 189ms/step - loss: 0.5287 - acc: 0.7891 - sparse_categorical_accuracy: 0.7891 - val_loss: 0.4944 - val_acc: 0.8006 - val_sparse_categorical_accuracy: 0.8006


In [26]:
result = model_bert_base_cased.evaluate(test_dataset_batch)

1225/1225 [==============================] - 75s 61ms/step - loss: 2.1504 - acc: 0.3529 - sparse_categorical_accuracy: 0.3529


In [27]:
result = model_bert_base_cased.evaluate(validation_dataset_batch)

1227/1227 [==============================] - 77s 63ms/step - loss: 0.4944 - acc: 0.8006 - sparse_categorical_accuracy: 0.8006


In [28]:
test_predict = model_bert_base_cased.predict(validation_dataset_batch.take(1))
test_predict

TFSequenceClassifierOutput(loss=None, logits=array([[-0.29195213,  1.557335  , -0.7351178 ],
       [ 2.3099828 , -0.5003828 , -1.5834221 ],
       [-3.0622053 , -1.8030615 ,  4.020719  ],
       [ 2.932993  , -1.0716059 , -1.8001673 ],
       [-2.1897635 , -1.7420418 ,  2.9177992 ],
       [-1.9516946 , -0.07686663,  1.578659  ],
       [-1.2401516 ,  1.4058137 ,  0.16447748],
       [ 2.858041  , -1.1485687 , -1.7349013 ]], dtype=float32), hidden_states=None, attentions=None)

validation set에 대한 최종 accaurcy가 `0.8006`인 반면 test set에 대한 최종 accuracy는 `0.3529`를 기록하였다.

### 모델 학습 2 - TFBertForSequenceClassification(bert-base-cased)

데이터에 좀 더 robuster 할 수 있도록하는 방안을 생각하던 중 dropout을 증가시켜 보면 어떨까하는 것이 떠올랐다. defalut 값이 0,1인 dropout 값을 0.3으로 증가시켜 학습을 진행하여 보았다.

In [32]:
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=0, verbose=0, mode='auto')

In [33]:
from transformers import TFBertForSequenceClassification
model_bert_base_cased_dr_3 = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', 
                                                                                    num_labels=num_classes, 
                                                                                    hidden_dropout_prob=0.3,
                                                                                    attention_probs_dropout_prob=0.3)
model_bert_base_cased_dr_3.compile(optimizer=optimizer, loss=loss, metrics=['acc'])
model_bert_base_cased_dr_3

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [36]:
# 이전 스텝에서 배치처리를 진행한 데이터셋(xxxx_dataset_batch)을 활용
history = model_bert_base_cased_dr_3.fit(train_dataset_batch, 
                                         epochs=20, 
                                         steps_per_epoch=1000, 
                                         validation_data=validation_dataset_batch,
                                         callbacks=[early_stopping])

Epoch 1/20
1000/1000 [==============================] - 253s 253ms/step - loss: 0.7369 - acc: 0.6852 - val_loss: 0.7260 - val_acc: 0.7125
Epoch 2/20
1000/1000 [==============================] - 247s 247ms/step - loss: 0.7309 - acc: 0.6949 - val_loss: 0.6897 - val_acc: 0.7205
Epoch 3/20
1000/1000 [==============================] - 253s 253ms/step - loss: 0.7327 - acc: 0.6877 - val_loss: 0.6838 - val_acc: 0.7338
Epoch 4/20
1000/1000 [==============================] - 254s 254ms/step - loss: 0.7222 - acc: 0.6967 - val_loss: 0.6593 - val_acc: 0.7412
Epoch 5/20
1000/1000 [==============================] - 255s 255ms/step - loss: 0.7234 - acc: 0.6951 - val_loss: 0.6534 - val_acc: 0.7475
Epoch 6/20
1000/1000 [==============================] - 256s 256ms/step - loss: 0.6930 - acc: 0.7076 - val_loss: 0.6028 - val_acc: 0.7574
Epoch 7/20
1000/1000 [==============================] - 254s 254ms/step - loss: 0.6777 - acc: 0.7197 - val_loss: 0.5995 - val_acc: 0.7616
Epoch 8/20
1000/1000 [============

In [37]:
result_3 = model_bert_base_cased_wich_config.evaluate(test_dataset_batch)

1225/1225 [==============================] - 76s 62ms/step - loss: 1.6921 - acc: 0.2710 - sparse_categorical_accuracy: 0.2710


validation set에 대한 최종 accaurcy가 `0.7623`, test set에 대한 accuracy는 `0.2710`을 기록하였다. 학습 epoch를 더 늘린다면 이 방법이 더 좋을 수 있지 않을까?하는 생각은 들었지만 현재 상태로는 모델의 학습결과가 이전에 비해 개선되지 않았다.

## Step 5. 결론
---

### 프로젝트 평가

1. MNLI 데이터셋을 처리하는 전용 Processor 클래스를 정상적으로 구현하였다. <br>Processor 클래스에 대해 1개 이상의 example에 대한 단위테스트가 정상 진행되었다. 
    - 3개의 클래스를 가진 데이터셋에 대하여 Processor 클래스가 정상적으로 작동 되었다.
2. BERT tokenizer와 Processor를 결합하여 데이터셋을 정상적으로 생성하였다. <br>MNLI 데이터셋의 입력과 라벨의 정의에 잘 맞는 tf.data.Dataset 인스턴스가 얻어졌다.
    - 데이터셋이 정상적으로 생성되었다.
3. MNLI 데이터셋에 대해 BERT 모델을 fine-tuning한 결과 기대한 수준의 모델 성능을 확인하였다.<br>테스트셋에 대해 80% 이상의 accuracy를 확인하였다. 
    - 최고 test accruacy는 0.3529로 기준에 미치지 못하였다.
    - 다만 validation set에 대한 accuracy는 0.8006을 기록하였다.